# Optimize Ramp Settings

This notebook outlines an example to optimize the ramp settings for a few different types of observations.

In these types of optimizations, we must consider observations constraints such as saturation levels, SNR requirements, and limits on acquisition time.

**Note**: The reported acquisition time does not include obsevatory and instrument-level overheads, such as slew times, filter changes, script compilations, etc. It only includes detector readout times (including reset frames).

In [1]:
## Standard imports

# Makes print and division act like Python 3
from __future__ import print_function, division

# Import the usual libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Enable inline plotting at lower left
%matplotlib inline
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'none'

# seaborn package for making pretty plots, but not necessary
try:
    import seaborn as sns
    params =   {'xtick.direction': 'in', 'ytick.direction': 'in', 'font.family': ['serif'],
                'text.usetex': True, 'text.latex.preamble': ['\usepackage{gensymb}']}
    sns.set_style("ticks", params)
except ImportError:
    print('Seaborn module is not installed. Not strictly required, but is a useful package!')
    
from IPython.display import display, Latex, clear_output

In [2]:
import pynrc
from pynrc import nrc_utils
from pynrc.nrc_utils import (webbpsf, poppy, pix_noise, S)
from pynrc.pynrc_core import table_filter

pynrc.setup_logging('WARNING', verbose=False)

from astropy.table import Table

## Example 1: M-Dwarf companion

We want to observe an M-Dwarf companion (K=18 mag) orbiting an F0V primary (K=10 mag) in the F430M filter. Assume the M-Dwarf flux is not significantly impacted by the primary's PSF (ie., in the background limited regime). In this scenario, the F0V star will saturate much more quickly compared to the fainter companion, so it limits which ramp settings we can use.

We will test a couple different types of observations (direct imaging vs coronagraphy).

In [3]:
# Get stellar spectra and normalize at K-Band
# The stellar_spectrum convenience function creates a Pysynphot spectrum
bp_k = S.ObsBandpass('k')
sp_M2V = pynrc.stellar_spectrum('M2V', 18, 'vegamag', bp_k)
sp_F0V = pynrc.stellar_spectrum('F0V', 10, 'vegamag', bp_k)

In [5]:
# Initiate a NIRCam observation
nrc = pynrc.NIRCam('F430M', wind_mode='WINDOW', xpix=160, ypix=160)

# What constraints do we place on the observations?
# Let's assume we want photometry on the primary to calibrate the M-Dwarf
#  - Set well_frac_max=0.75
# Want a SNR~100 in the F430M filter
#  - Set snr_goal=100
res = nrc.ramp_optimize(sp_M2V, sp_bright=sp_F0V, snr_goal=100, well_frac_max=0.75, verbose=True)

BRIGHT1
BRIGHT2
DEEP2
DEEP8
MEDIUM2
MEDIUM8
RAPID
SHALLOW2
SHALLOW4
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
RAPID         9  510      2.51   1278.96   1421.06    100.0    0.707    2.651
RAPID         9  511      2.51   1281.47   1423.85    100.1    0.707    2.651
RAPID         9  512      2.51   1283.97   1426.64    100.2    0.707    2.651
RAPID         9  513      2.51   1286.48   1429.42    100.3    0.707    2.651
RAPID         9  514      2.51   1288.99   1432.21    100.3    0.707    2.651
RAPID         9  515      2.51   1291.50   1435.00    100.4    0.707    2.651
RAPID         9  516      2.51   1294.00   1437.78    100.5    0.707    2.651
RAPID         9  517      2.51   1296.51   1440.57    100.6    0.707    2.651
RAPID         9  518      2.51   1299.02   1443.36    100.7    0.707    2.651
RAPID         9  519      2.51   1301.53   1446.14    100.8    0.707    2.

In [49]:
# Print the Top 2 settings for each readout pattern
res2 = table_filter(res, 2)
print(res2)

 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
RAPID         9  510      2.51   1278.96   1421.06    100.0    0.707    2.651
RAPID         9  511      2.51   1281.47   1423.85    100.1    0.707    2.651
BRIGHT1       5  700      2.51   1755.43   1950.48     99.9    0.707    2.262
BRIGHT1       5  701      2.51   1757.94   1953.27    100.0    0.707    2.262
BRIGHT2       4  751      2.23   1674.07   1883.33    100.0    0.628    2.303
BRIGHT2       4  752      2.23   1676.30   1885.84    100.0    0.628    2.303
SHALLOW4      2  621      2.51   1557.32   1730.35    100.0    0.707    2.403
SHALLOW4      2  622      2.51   1559.83   1733.14    100.1    0.707    2.403


In [46]:
# Do the same thing, but for coronagraphic mask instead
nrc._mask = 'MASK430R'; nrc._pupil = 'CIRCLYOT'
nrc.update_psf_coeff()
nrc.update_detectors(xpix=320, ypix=320)

# We assume that longer ramps will give us the best SNR for time
patterns = ['MEDIUM8', 'DEEP8']
res = nrc.ramp_optimize(sp_M2V, sp_bright=sp_F0V, snr_goal=100, well_frac_max=0.75, patterns=patterns)

# Take the Top 2 settings for each readout pattern
res2 = table_filter(res, 2)
print(res)

 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
MEDIUM8      10   83    104.77   8695.57   8784.30    100.0    0.001    1.066
MEDIUM8      10   84    104.77   8800.34   8890.14    100.6    0.001    1.066
DEEP8        20   12    414.79   4977.45   4990.28     99.0    0.003    1.400
DEEP8        20   13    414.79   5392.24   5406.14    103.0    0.003    1.400


### RESULTS

Based on these two comparisons, it looks like direct imaging is much more efficient in getting to the requisite SNR. In addition, direct imaging gives us a photometric comparison source that the coronagraph masks out.

## Example 2: Single-Object Grism Spectroscopy



In [3]:
# M9V star at K=10 mags
bp_k = S.ObsBandpass('k')
sp_M9V = pynrc.stellar_spectrum('M9V', 10, 'vegamag', bp_k)

In [4]:
nrc = pynrc.NIRCam('F444W', pupil='GRISM0', wind_mode='STRIPE', ypix=64)

In [5]:
res = nrc.ramp_optimize(sp_M9V, snr_goal=300, nint_min=5, verbose=True)

BRIGHT1
BRIGHT2
DEEP2
DEEP8
MEDIUM2
MEDIUM8
RAPID
SHALLOW2
SHALLOW4
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
MEDIUM2       8    5     24.52    122.62    124.32    301.9    0.114   27.080
MEDIUM2       8    6     24.52    147.14    149.19    330.8    0.114   27.080
MEDIUM2       7    6     21.12    126.71    128.75    304.9    0.098   26.867
DEEP2         4    6     21.12    126.71    128.75    302.9    0.098   26.696
DEEP2         4    7     21.12    147.82    150.21    327.2    0.098   26.696
SHALLOW2     10    8     16.01    128.07    130.79    304.2    0.074   26.602
SHALLOW2     10    9     16.01    144.08    147.14    322.7    0.074   26.602
MEDIUM2       6    8     17.71    141.69    144.42    319.3    0.082   26.566
SHALLOW2      9   10     14.31    143.06    146.46    319.2    0.066   26.375
MEDIUM8       8    5     26.57    132.84    134.54    305.2    0.123   26.

In [6]:
# Print the Top 2 settings for each readout pattern
res2 = table_filter(res, 2)
print(res2)

 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
RAPID        10   52      3.41    177.12    194.83    298.4    0.016   21.378
RAPID        10   53      3.41    180.52    198.58    301.3    0.016   21.378
BRIGHT1      10   22      6.47    142.37    149.87    296.6    0.030   24.225
BRIGHT1      10   23      6.47    148.85    156.68    303.2    0.030   24.225
BRIGHT2      10   21      6.81    143.06    150.21    301.0    0.032   24.559
BRIGHT2      10   22      6.81    149.87    157.36    308.1    0.032   24.559
SHALLOW2     10    8     16.01    128.07    130.79    304.2    0.074   26.602
SHALLOW2     10    9     16.01    144.08    147.14    322.7    0.074   26.602
SHALLOW4      8   10     13.28    132.84    136.24    300.8    0.062   25.767
SHALLOW4      8   11     13.28    146.12    149.87    315.4    0.062   25.767
MEDIUM2       8    5     24.52    122.62    124.32    301.9    0

In [5]:
# Let's say we choose SHALLOW4, NGRP=8, NINT=10
# Update detector readout
nrc.update_detectors(read_mode='SHALLOW4', ngroup=8, nint=10)

# Print final wavelength-dependent SNR
# For spectroscopy, the snr_goal is the average over the bandpass
snr_dict = nrc.sensitivity(sp=sp_M9V, forwardSNR=True, units='mJy', verbose=True)

F444W SNR for M9V source
   Wave      SNR    Flux (mJy)
--------- --------- ----------
     3.90    419.63      34.57
     4.00    425.97      30.34
     4.10    380.28      23.59
     4.20    390.77      27.64
     4.30    365.56      26.10
     4.40    320.72      21.98
     4.50    292.65      19.63
     4.60    264.16      18.21
     4.70    215.66      14.01
     4.80    208.18      14.47
     4.90    225.64      21.20
     5.00     99.98      11.15


## Example 3: Exoplanet Coronagraphy


## Example 4: Exoplanet Transit Spectroscopy

Let's say we want to observe an exoplanet transit using NIRCam grisms in the F322W2 filter. Given that the transit duration is 2 hours for the source, what is the best 

In [3]:
nrc = pynrc.NIRCam('F322W2', pupil='GRISM0', wind_mode='STRIPE', ypix=64)

In [4]:
# K6V star at K=8.4 mags
bp_k = S.ObsBandpass('k')
sp_K6V = pynrc.stellar_spectrum('K6V', 8.4, 'vegamag', bp_k)

In [13]:
# Constraints
well     = 0.5      # keep well below 50% full
tacq     = 2*3600.  # 2 hour transit duration
ng_max   = 30       # transit spec allows up to 30 groups per integrations
nint_max = int(1e6) # Effectively no limit on number of integrations

# Let's bin the spectrum to R~100
R = 100
dw_bin = (nrc.bandpass.avgwave() / 10000) / R

res = nrc.ramp_optimize(sp_K6V, tacq_max=tacq, nint_max=nint_max, ng_max=ng_max, well_frac_max=well,
                        dw_bin=dw_bin, verbose=True)

BRIGHT1
BRIGHT2
DEEP2
DEEP8
MEDIUM2
MEDIUM8
RAPID
SHALLOW2
SHALLOW4
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
BRIGHT1      24  438     16.01   7011.80   7160.98  28100.1    0.499  332.064
BRIGHT1      24  439     16.01   7027.81   7177.33  28132.2    0.499  332.064
BRIGHT1      24  440     16.01   7043.81   7193.68  28164.2    0.499  332.064
BRIGHT1      24  441     16.01   7059.82   7210.03  28196.2    0.499  332.064
BRIGHT1      24  442     16.01   7075.83   7226.38  28228.2    0.499  332.064
BRIGHT1      23  457     15.33   7004.64   7160.30  28067.0    0.478  331.688
BRIGHT2      23  448     15.67   7019.29   7171.88  27819.8    0.489  328.501
BRIGHT2      23  449     15.67   7034.96   7187.89  27850.8    0.489  328.501
BRIGHT2      23  450     15.67   7050.63   7203.90  27881.8    0.489  328.501
BRIGHT2      23  451     15.67   7066.30   7219.91  27912.8    0.489  328.

In [14]:
# Print the Top 2 settings for each readout pattern
res2 = table_filter(res, 2)
print(res2)

 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
RAPID        30  680     10.22   6948.44   7180.06  27779.1    0.319  327.833
RAPID        30  681     10.22   6958.66   7190.62  27799.5    0.319  327.833
BRIGHT1      24  438     16.01   7011.80   7160.98  28100.1    0.499  332.064
BRIGHT1      24  439     16.01   7027.81   7177.33  28132.2    0.499  332.064
BRIGHT2      23  448     15.67   7019.29   7171.88  27819.8    0.489  328.501
BRIGHT2      23  449     15.67   7034.96   7187.89  27850.8    0.489  328.501
SHALLOW2     10  438     16.01   7011.80   7160.98  27786.4    0.499  328.356
SHALLOW2     10  439     16.01   7027.81   7177.33  27818.1    0.499  328.356
SHALLOW4      9  468     14.99   7013.84   7173.25  27103.3    0.467  320.010
SHALLOW4      9  469     14.99   7028.83   7188.57  27132.2    0.467  320.010
MEDIUM2       5  490     14.31   7009.75   7176.65  27657.7    0

In [15]:
# Choose RAPID, ngroup=30, nint=683
nrc.update_detectors(read_mode='RAPID', ngroup=30, nint=683)
_ = nrc.sensitivity(sp=sp_K6V, dw_bin=dw_bin, forwardSNR=True, verbose=True, units='Jy')

F322W2 SNR for K6V source
   Wave      SNR    Flux (Jy)
--------- --------- ---------
     2.40   5277.42      0.25
     2.50  27466.94      0.21
     2.60  30416.54      0.22
     2.70  31050.02      0.20
     2.80  32232.72      0.20
     2.90  31706.15      0.19
     3.00  30153.86      0.17
     3.10  28830.42      0.17
     3.20  30286.46      0.16
     3.30  31074.73      0.15
     3.40  30451.30      0.15
     3.50  29931.43      0.14
     3.60  29448.36      0.13
     3.70  28795.29      0.12
     3.80  27828.64      0.12
     3.90  26546.33      0.12
     4.00  21788.15      0.11


## Example 5: Extended Souce

